Función para limpieza de categorías

In [2]:
import pandas as pd
from merges_tk import RestaurantDataCleaner, FuzzyMatcher, DataMerger
from geopy.geocoders import Nominatim
import pandas as pd
import time

# Crear un geolocalizador
geolocator = Nominatim(user_agent="geoapiExercises")

def main():
    # Cargar datos
    df_yelp = pd.read_parquet("Data_cleaned/Yelp/filtered_restaurants.parquet")
    df_google = pd.read_parquet("Data_cleaned/Google/metadata-sitios/filtered_top_10_states.parquet")
    
    # Configurar y ejecutar el proceso de merge
    cleaner = RestaurantDataCleaner()
    matcher = FuzzyMatcher()
    merger = DataMerger(cleaner, matcher)
    
    result = merger.merge_datasets(df_google, df_yelp)
    return result.drop(columns = ["city"])

if __name__ == "__main__":
    result_df = main()

h:\restaurantguru\.venv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
INFO:merges_tk:Encontrados 7856 pares dentro del umbral de distancia
INFO:merges_tk:Encontradas 1297 coincidencias finales


In [3]:
import pandas as pd
from scipy.spatial import cKDTree
import numpy as np

class LocalGeocoder:
    def __init__(self):
        # Cargar base de datos de ciudades (ejemplo con worldcities.csv de SimpleMaps)
        self.cities_df = pd.read_csv('uscities.csv')
        self.kdtree = cKDTree(np.deg2rad(self.cities_df[['lat', 'lng']].values))
    
    def find_nearest_city(self, lat, lon, max_distance_km=50):
        # Convertir a radianes
        query_point = np.deg2rad(np.array([[lat, lon]]))
        
        # Encontrar el vecino más cercano
        distance, index = self.kdtree.query(query_point, k=1)
        
        # Convertir distancia a kilómetros (radio de la Tierra ≈ 6371 km)
        distance_km = 6371 * distance[0]
        
        if distance_km <= max_distance_km:
            return self.cities_df.iloc[index[0]]['city']
        return None

# Uso
geocoder = LocalGeocoder()
result_df['city'] = result_df.apply(
    lambda row: geocoder.find_nearest_city(row['latitude'], row['longitude']), 
    axis=1
)

In [4]:
result_df

,id_G,id_Y,name,address,state,postal_code,latitude,longitude,hours,categories,city
0,0x88c2c5c515fa6e53:0xee4f630ca314fe01,O1x70YS930qP3beQIystoA,Flock and Stock,615 Channelside Dr,FL,33602,27.943383,-82.447510,"{'Friday': '11:0-23:0', 'Monday': None, 'Satur...","[Restaurants, Southern, Burgers]",Tampa
1,0x886466970c8a4e45:0xb01d015d268be1be,LIVL3r0LDrKl2RwbQ6TY8w,GoGo Sushi,2057 Scarritt Pl,TN,37212,36.145913,-86.799029,"{'Friday': '11:0-22:0', 'Monday': '11:0-21:0',...","[Restaurants, Sushi Bars, Japanese]",Nashville
2,0x89c6c99fe6b8db97:0x15ade9e74de1f26d,lsDw9DHpC2XADe5rvxrRQQ,Airis Roll & Rice,8 N 3rd St,PA,19106,39.950497,-75.145778,"{'Friday': '11:0-20:0', 'Monday': '11:0-20:0',...","[Poke, Japanese, Sushi Bars, Food, Restaurants]",Camden
3,0x88644383d230fe1d:0x3a533684a5378a4f,8VOAQKQkYJsKrkFfCXkP_g,Chipotle Mexican Grill,1806 Gallatin Pike N,TN,37115,36.298881,-86.698047,"{'Friday': '10:45-22:0', 'Monday': '10:45-22:0...","[Fast Food, Mexican, Restaurants]",Goodlettsville
4,0x89c6c18255900fa9:0x65a6ed6f64d5ee46,o8aFtFrZcWp-twnIgdqwyQ,Wawa,525 Burmont Rd,PA,19026,39.946901,-75.301571,"{'Friday': '0:0-0:0', 'Monday': '0:0-0:0', 'Sa...","[Sandwiches, Coffee & Tea, Restaurants, Delis,...",Drexel Hill
...,...,...,...,...,...,...,...,...,...,...,...
71980,0x89c81997ab131515:0xee746ac21887fe01,NaN,Liberty Pizza & Sub,8136 Liberty Road Unit #G,MD,21244,39.355377,-76.761759,"[[Saturday, 11AM–4AM], [Sunday, 11AM–4AM], [Mo...",[Restaurant],Milford Mill
71981,0x89c81bdd8ae87749:0xd0c17064d8560660,NaN,Melt,6325 Windsor Mill Rd # 1,MD,21207,39.321513,-76.726280,"[[Saturday, 10AM–10PM], [Sunday, 11AM–4PM], [M...",[Pizza restaurant],Lochearn
71982,0x89d31f176b64da79:0x202faca0f650e880,NaN,China King Express,6938 Erie Rd,NY,14047,42.698185,-78.988253,"[[Saturday, 11AM–11PM], [Sunday, 11AM–11PM], [...",[Chinese restaurant],Highland-on-the-Lake
71983,0x89d373386b718fff:0xf82c90ef48e7b23,NaN,House of Gourmet -- 食全食美,2865 Sheridan Dr Suite B,NY,14150,42.980251,-78.827337,"[[Saturday, 11:15AM–9:45PM], [Sunday, 11:15AM–...","[Chinese restaurant, Sichuan restaurant]",Eggertsville


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import sys
import os

# Agregar la carpeta raíz (un nivel arriba de EDA) a sys.path
root_path = os.path.abspath(os.path.join(".."))
if root_path not in sys.path:
    sys.path.append(root_path)
from clean_functions.clean_metadatos import process_metadata_files, clean_and_merge_reviews

warnings.filterwarnings("ignore")


In [3]:
input_directory = "Datasets/Google/metadata-sitios"
output_path = "Data_cleaned/Google/metadata-sitios/filtered_restaurants.parquet"
process_metadata_files(input_directory, output_path)


Archivo exportado exitosamente en Data_cleaned/Google/metadata-sitios/filtered_restaurants.parquet


In [3]:
df_completo = pd.read_parquet('Data_cleaned/Google/metadata-sitios/filtered_restaurants.parquet')

In [4]:
df_completo.sort_values(by="city")

,gmap_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,categories,hours
180282,0x862367660d3744f7:0x4b8cdb92355483b1,Sweet Something Yogurt Shop,"Sweet Something Yogurt Shop, 2780 Rodeo Rd, Ab...",Abbeville,LA,70510,29.974725,-92.102700,5.0,8,[Restaurant],"[[Sunday, 1–5PM], [Monday, 1–8PM], [Tuesday, 1..."
180752,0x888d5fcab4c942cb:0x5c826553422d5c4f,Kava Coffee Exchange,"Kava Coffee Exchange, 106 Kirkland St, Abbevil...",Abbeville,AL,36310,31.570603,-85.250588,5.0,38,"[Restaurant, Coffee shop]","[[Sunday, Closed], [Monday, 7AM–5PM], [Tuesday..."
22980,0x862367ef0ba1f5b3:0x9b2afc5eaeb6b54,King's Seafood Express,"King's Seafood Express, 3011 Veterans Memorial...",Abbeville,LA,70510,29.970367,-92.093529,4.0,18,[Seafood restaurant],"[[Friday, 10AM–9:30PM], [Saturday, 10AM–9:30PM..."
16278,0x86236800a2318681:0x9a3e210e3e3b8f01,Amore Pizzaroma,"Amore Pizzaroma, 301 Ardly Dr, Abbeville, LA 7...",Abbeville,LA,70510,29.973688,-92.103767,4.5,206,"[Pizza restaurant, Italian restaurant, Restaur...","[[Friday, 11AM–9PM], [Saturday, 11AM–9PM], [Su..."
16279,0x862367f2204bcb53:0x39ef1081b43ef18,Midway on the Square Italian Steakhouse,"Midway on the Square Italian Steakhouse, 116 N...",Abbeville,LA,70510,29.974312,-92.136518,4.4,76,[Italian restaurant],"[[Friday, 11AM–10PM], [Saturday, 5–10PM], [Sun..."
...,...,...,...,...,...,...,...,...,...,...,...,...
211698,0x884245a83f101f61:0x5e4527a22ec49915,Bazaar Eatery,None,None,None,None,38.028330,-84.471565,4.5,18,[Restaurant],"[[Saturday, 11AM–10PM], [Sunday, 11AM–6PM], [M..."
211864,0x89ac8f0e0d23ad2b:0x908634c7562e94,King Wok,"King Wok, 7405 Sunset Lake Rd, Fuquay-Varina, ...",None,None,None,35.595917,-78.774826,3.9,118,"[Chinese restaurant, Asian restaurant]","[[Saturday, 12–8PM], [Sunday, 12–8PM], [Monday..."
211899,0x88deef8a6a84cb87:0x40984bb82fc8c727,Super Taquerias,"Super Taquerias, 8031 S Federal Hwy, Port St. ...",None,None,None,27.319077,-80.316980,4.8,36,[Restaurant],"[[Saturday, 11AM–9PM], [Sunday, 11AM–9PM], [Mo..."
211935,0x88dd4720714d6511:0xbd58b5b898f89fe0,The Arepazo Grill,None,None,None,None,28.062436,-81.947359,4.6,48,"[Fast food restaurant, Venezuelan restaurant]","[[Saturday, 3–11PM], [Sunday, 11AM–7PM], [Mond..."


In [5]:
df_completo.dropna(subset=['city'],ignore_index= True ,inplace=True)
df_completo.sort_values(by="city")

,gmap_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,categories,hours
169217,0x862367660d3744f7:0x4b8cdb92355483b1,Sweet Something Yogurt Shop,"Sweet Something Yogurt Shop, 2780 Rodeo Rd, Ab...",Abbeville,LA,70510,29.974725,-92.102700,5.0,8,[Restaurant],"[[Sunday, 1–5PM], [Monday, 1–8PM], [Tuesday, 1..."
169593,0x888d5fcab4c942cb:0x5c826553422d5c4f,Kava Coffee Exchange,"Kava Coffee Exchange, 106 Kirkland St, Abbevil...",Abbeville,AL,36310,31.570603,-85.250588,5.0,38,"[Restaurant, Coffee shop]","[[Sunday, Closed], [Monday, 7AM–5PM], [Tuesday..."
22620,0x862367ef0ba1f5b3:0x9b2afc5eaeb6b54,King's Seafood Express,"King's Seafood Express, 3011 Veterans Memorial...",Abbeville,LA,70510,29.970367,-92.093529,4.0,18,[Seafood restaurant],"[[Friday, 10AM–9:30PM], [Saturday, 10AM–9:30PM..."
16018,0x86236800a2318681:0x9a3e210e3e3b8f01,Amore Pizzaroma,"Amore Pizzaroma, 301 Ardly Dr, Abbeville, LA 7...",Abbeville,LA,70510,29.973688,-92.103767,4.5,206,"[Pizza restaurant, Italian restaurant, Restaur...","[[Friday, 11AM–9PM], [Saturday, 11AM–9PM], [Su..."
16019,0x862367f2204bcb53:0x39ef1081b43ef18,Midway on the Square Italian Steakhouse,"Midway on the Square Italian Steakhouse, 116 N...",Abbeville,LA,70510,29.974312,-92.136518,4.4,76,[Italian restaurant],"[[Friday, 11AM–10PM], [Saturday, 5–10PM], [Sun..."
...,...,...,...,...,...,...,...,...,...,...,...,...
105098,0x8630a94cfb46adc9:0xbf21e7a339756e76,Casa Michoacána Mexican Restaurant,"Casa Michoacána Mexican Restaurant, 2243 Port ...",Zwolle,LA,71486,31.631761,-93.642246,4.2,27,[Mexican restaurant],"[[Sunday, 11AM–9PM], [Monday, 11AM–9PM], [Tues..."
3300,0x8630a9327e80eccb:0x2d72e395632be761,Frazier's BBQ,"Frazier's BBQ, 2325-2245 Port Arthur St, Zwoll...",Zwolle,LA,71486,31.631139,-93.642566,4.6,5,[Barbecue restaurant],"[[Monday, Closed], [Tuesday, Closed], [Wednesd..."
53089,0x89c25f7e8d96cca3:0xf6a6a93ec95900fa,Yun Cafe & Asian Mart,"Yun Cafe & Asian Mart, 7305 37th Road Store #2...",lower level,NY,11372,40.747453,-73.892182,4.8,18,[Burmese restaurant],"[[Tuesday, 7:30AM–9PM], [Wednesday, 7:30AM–9PM..."
4007,0x89c259e633c5b551:0x4143b924418b7def,McPutins Chicken,"McPutins Chicken, new york, NY 10019",new york,NY,10019,40.763163,-73.984732,4.6,8,"[Fried chicken takeaway, Chicken restaurant, E...","[[Sunday, 3–11:30PM], [Monday, 3–11:30PM], [Tu..."
